# <b> <p align="center"> <span style="color: #DCC43C"> PYTORCH EXPERIMENT TRACKING <span> </p> </b>
### <b> <p align="center"> <span style="color: #BFF0FF"> let's explore <span> </p> </b>

Machine learning is experimental . 

To know which experments are worth pursuing that's where **Experimental tracking** comes in to figure out what doesn't work so you can figure out what does work.

In this notebook, we're going to see an example fo programmatically tracking experiments

And so far we've keep track of them via Python dictionaries.

Or just comparing them by the metric print outs during training.

What if you wanted to run a dozen (or more) different models at once?

Surely there's a better way...

There is.

**Experiment tracking.**

And since experiment tracking is so important and integral to machine learning, you can consider this notebook your first milestone project.

So welcome to Milestone Project 1: FoodVision Mini Experiment Tracking.

We're going to answer the question: **how do I track my machine learning experiments?**

## Why track experiments?

If you're only running a handful of models (like we've done so far), it might be okay just to track their results in print outs and a few dictionaries.

However, as the number of experiments you run starts to increase, this naive way of tracking could get out of hand.

So if you're following the machine learning practitioner's motto of *experiment, experiment, experiment!*, you'll want a way to track them.

<img src="https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/07-experiment-tracking-can-get-out-of-hand.png" alt="experiment tracking can get out of hand, many different experiments with different names" width=900/>

*After building a few models and tracking their results, you'll start to notice how quickly it can get out of hand.*

## Different ways to track machine learning experiments

There are as many different ways to track machine learning experiments as there is experiments to run.

This table covers a few.

| **Method** | **Setup** | **Pros** | **Cons** | **Cost** |
| ----- | ----- | ----- | ----- | ----- |
| Python dictionaries, CSV files, print outs | None | Easy to setup, runs in pure Python | Hard to keep track of large numbers of experiments | Free |
| [TensorBoard](https://www.tensorflow.org/tensorboard/get_started) | Minimal, install [`tensorboard`](https://pypi.org/project/tensorboard/) | Extensions built into PyTorch, widely recognized and used, easily scales. | User-experience not as nice as other options. | Free |
| [Weights & Biases Experiment Tracking](https://wandb.ai/site/experiment-tracking) | Minimal, install [`wandb`](https://docs.wandb.ai/quickstart), make an account | Incredible user experience, make experiments public, tracks almost anything. | Requires external resource outside of PyTorch. | Free for personal use |
| [MLFlow](https://mlflow.org/) | Minimal, install `mlflow` and starting tracking | Fully open-source MLOps lifecycle management, many integrations. | Little bit harder to setup a remote tracking server than other services. | Free |

<img src="https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/07-different-places-to-track-experiments.png" alt="various places to track machine learning experiments" width=900/>

*Various places and techniques you can use to track your machine learning experiments. **Note:** There are various other options similar to Weights & Biases and open-source options similar to MLflow but I've left them out for brevity. You can find more by searching "machine learning experiment tracking".*

## 0. Getting Setup 

Going moduler [section](https://github.com/ShafaetUllah032/DL_with_PyTorch/tree/main/going_modular) will be needed here.
We'll also get the [`torchinfo`](https://github.com/TylerYep/torchinfo) package if it's not available.

`torchinfo` will help later on to give us visual summaries of our model(s).

And since we're using a newer version of the `torchvision` package (v0.13 as of June 2022), we'll make sure we've got the latest versions.

In [5]:
import torch 
import torchvision

print(torch.__version__, torchvision.__version__)

2.3.0 0.18.0


As we have required version ... no need to download

If you have the lower version then, 

```
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

```

In [6]:
# Continue with regular imports
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary


Now let's setup device agnostic code.

> **Note:** If you're using Google Colab, and you don't have a GPU turned on yet, it's now time to turn one on via `Runtime -> Change runtime type -> Hardware accelerator -> GPU`.

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
device

'cuda'

### Create a helper function to set seeds

Since we've been setting random seeds a whole bunch throughout previous sections, how about we functionize it?

Let's create a function to "set the seeds" called `set_seeds()`.

> **Note:** Recall a [random seed](https://en.wikipedia.org/wiki/Random_seed) is a way of flavouring the randomness generated by a computer. They aren't necessary to always set when running machine learning code, however, they help ensure there's an element of reproducibility (the numbers I get with my code are similar to the numbers you get with your code). Outside of an education or experimental setting, random seeds generally aren't required.

In [8]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

## 1. Get data 

As always , before we can run machine learning experimnets, we will need a dataset

We're going to continue trying to improve upon the results we've been getting on Foobvision Mini.

In the last notebook we saw how powerful the transfer learning is. Let's do something expermnetal do improve the result.

Let's goo....



In [9]:
from going_modular import get_data

train_path,test_path=get_data.get_data(path="for experiment tracking",
                                       sub_folder="food_data",
                                       url="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
train_path, test_path

path_exists , skip creating .....


(WindowsPath('for experiment tracking/food_data/train'),
 WindowsPath('for experiment tracking/food_data/test'))

Now we got our data , let's go for experiments and track the experiments

## 2. Create Dataset and dataloader

As we have the data, to train and test , We have to convert them into the dataloader.

We can do by using the `create_dataloader` function from `going_moduler.datasetup` from [going moduder](https://github.com/ShafaetUllah032/DL_with_PyTorch/tree/main/going_modular)

And since we'll be using transfer learning and specifically pretrained models from [`torchvision.models`](https://pytorch.org/vision/stable/models.html), we'll create a transform to prepare our images correctly.

To transform our images in tensors, we can use:
1. Manually created transforms using `torchvision.transforms`.
2. Automatically created transforms using `torchvision.models.MODEL_NAME.MODEL_WEIGHTS.DEFAULT.transforms()`.
    * Where `MODEL_NAME` is a specific `torchvision.models` architecture, `MODEL_WEIGHTS` is a specific set of pretrained weights and `DEFAULT` means the "best available weights".
    
do did it at [transfer learning](https://github.com/ShafaetUllah032/DL_with_PyTorch/blob/main/06%20pytorch%20tranfer%20learning.ipynb) section

Let's see first an example of manually creating a `torchvision.transforms` pipeline (creating a transforms pipeline this way gives the most customization but can potentially result in performance degradation if the transforms don't match the pretrained model).

The main manual transformation we need to be sure of is that all of our images are normalized in ImageNet format (this is because pretrained `torchvision.models` are all pretrained on [ImageNet](https://www.image-net.org/)).

We can do this with:

```python
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
```

### 2.1 Create DataLoaders using manually created transforms


In [10]:
# import the module

from going_modular import data_setup
from torchvision import transforms

# Setup ImageNet normalization levels (turns all images into similar distribution as ImageNet)
normalize=transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Create transform pipeline manually
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])
print(f"Manually created transforms: {manual_transforms}")


train_dataloader,test_dataloader,class_names=data_setup.create_dataloaders(train_dir=train_path,
                                                                           test_dir=test_path,
                                                                           transform=manual_transforms,
                                                                           batch_size=32)

train_dataloader,test_dataloader,class_names


Manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


(<torch.utils.data.dataloader.DataLoader at 0x1a293ea35b0>,
 ['pizza', 'steak', 'sushi'])

### 2.1 Create DataLoaders using automatically created transforms

Datatransfomr and DataLoader created !

We are going to create dataloader using automatics process 
We can do this by first instantiating a set of pretrained weights (for example `weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT`)  we'd like to use and calling the `transforms()` method on it.

In [11]:
# setup pretrained weights (plenty of these available in torchvision.models)

weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT

# Get transform from weight(these are the transform that were used to obtain the weights)
automatic_transform=weights.transforms()
print(f"automatically created transforms :{automatic_transform}")

train_dataloader,test_dataloader,class_names=data_setup.create_dataloaders(train_dir=train_path,
                                                                           test_dir=test_path,
                                                                           transform=automatic_transform,
                                                                           batch_size=32)

train_dataloader,test_dataloader,class_names

automatically created transforms :ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x1a293ecb6d0>,
 ['pizza', 'steak', 'sushi'])

## 3. Getting a pretrained model , freezing the base layers and changing the classifier head

Before we run an track multiple modeling experiments, let's see what it's like to run and track a single one And since our data is ready, the next thing we'll need a model.
Let's download the pretrained weights for a `torchvision.models.efficientnet_b0()` model and prepare it for use with our own data.